## Introduction to Pandas

[Pandas](http://pandas.pydata.org/) is the essential data analysis library for Python programmers. It provides fast and flexible data structures built on top of [numpy](http://www.numpy.org/).

It is well suited to handle "tabular" data (that might be found in a spreadsheet), time series data, or pretty much anything you care to put in a matrix with rows and named columns.

It contains two primary data structures, the `Series` (1-dimensional) and the `DataFrame` (2-dimensional) as well as a host of convenience methods for loading and plotting data.

The main thing that makes pandas pandas is that all data is *intrinsically aligned*. That means each data structure, `DataFrame` or `Series` has something called an `Index` that links data values with a label. That link will always be there (unless you explicitly break or change it) and its what allows pandas to quickly and efficiently "do the right thing" when working with data.

In [1]:
# The canonical way to import pandas:
import pandas as pd
import numpy as np

## The Series Object

A `Series` is a one-dimensional array of indexed data.

In [2]:
data = pd.Series([0.1, 0.2, 0.3, 0.4])
data

0    0.1
1    0.2
2    0.3
3    0.4
dtype: float64

The `Series` wraps a 1-d `ndarray` from numpy and an `Index` object.

In [3]:
data.values

array([ 0.1,  0.2,  0.3,  0.4])

In [4]:
data.index

RangeIndex(start=0, stop=4, step=1)

In [5]:
# This particular index type, the `RangeIndex`, lets us use the
# same square-bracket notation as a `ndarray` to access elements:
data[0]

0.10000000000000001

In [6]:
# or even a slice:
data[1:3]

1    0.2
2    0.3
dtype: float64

We don't have to use this auto-generated list of integers as the index though. Index values can be specified manually and don't even have to be integers.

In [7]:
data = pd.Series([0.1, 0.2, 0.3, 0.4], index=['a', 'b', 'c', 'd'])
data

a    0.1
b    0.2
c    0.3
d    0.4
dtype: float64

In [8]:
data.index

Index(['a', 'b', 'c', 'd'], dtype='object')

In [9]:
# Item access works just like before, with square brackets, 
# even though the index values are strings
data['a']

0.10000000000000001

In [10]:
# slices still work! But note the last element is included this time.
# This is the default behavior for indexes.
data['a':'c']

a    0.1
b    0.2
c    0.3
dtype: float64

In [11]:
# We could create a non-sequential integer index:
data = pd.Series([0.1, 0.2, 0.3, 0.4], index=[5, 8, 2, 1])
data

5    0.1
8    0.2
2    0.3
1    0.4
dtype: float64

In [12]:
data.index

Int64Index([5, 8, 2, 1], dtype='int64')

In [13]:
# Why?
data[1]

0.40000000000000002

Above we see the critical difference between numpy arrays, which are always ordered sequentially and have an implicit integer index, and `Series` objects, which have an index that maps *labels* to *values*.

`Series` are in fact a cross between a numpy array and a python dictionary. You can think of them as a dictionary with *typed* keys and *typed* values.

In [14]:
max_depths_dict = {
    'Erie': 64,
    'Huron': 229,
    'Michigan': 281,
    'Ontario': 244,
    'Superior': 406,
}
max_depths = pd.Series(max_depths_dict)
max_depths

Erie         64
Huron       229
Michigan    281
Ontario     244
Superior    406
dtype: int64

In [15]:
# squint and it looks like a dictionary!
max_depths['Michigan']

281

In [16]:
# Notice the index in this case was constructed automatically
# from the dictionary keys.
max_depths.index

Index(['Erie', 'Huron', 'Michigan', 'Ontario', 'Superior'], dtype='object')

We can think of an `Index` as an *immutable*, n-dimensional array. 

## The DataFrame Object

Much like the `Series` is a one-dimensional array of indexed data, a `DataFrame` is a two-dimensional array of indexed data.

You can think of a `DataFrame` as a sequence of `Series` objects all sharing the same index.

In [17]:
avg_depths_dict = {
    'Erie': 19,
    'Huron': 59,
    'Michigan': 85,
    'Ontario': 86,
    'Superior': 149,
}

avg_depths = pd.Series(avg_depths_dict)

lakes = pd.DataFrame({'Max Depth (m)': max_depths, 'Avg Depth (m)': avg_depths})
lakes

,Avg Depth (m),Max Depth (m)
Erie,19,64
Huron,59,229
Michigan,85,281
Ontario,86,244
Superior,149,406


In [18]:
# Just like the `Series`, a `DataFrame` has an `index` property
lakes.index

Index(['Erie', 'Huron', 'Michigan', 'Ontario', 'Superior'], dtype='object')

In [19]:
# and a `values` property that exposes the underlying `ndarray`
lakes.values

array([[ 19,  64],
       [ 59, 229],
       [ 85, 281],
       [ 86, 244],
       [149, 406]], dtype=int64)

In [21]:
# And unlike the Series, the DataFrame has a `columns` property
lakes.columns

Index(['Avg Depth (m)', 'Max Depth (m)'], dtype='object')

In [23]:
# We can get the shape of a dataframe, just like a numpy ndarray
lakes.shape

(5, 2)

In [24]:
# We can do dictionary-style lookups into the dataframe by column name
# to get a single Series:
lakes['Max Depth (m)']

Erie         64
Huron       229
Michigan    281
Ontario     244
Superior    406
Name: Max Depth (m), dtype: int64

In [25]:
# Or use slice syntax to select multiple columns.
lakes['Avg Depth (m)':'Max Depth (m)']

,Avg Depth (m),Max Depth (m)
Erie,19,64
Huron,59,229


### Creating new columns

Once we have a `DataFrame`, creating new columns is done through simple assignment.

In [26]:
surface_area = pd.Series({
    'Superior': 82097,
    'Michigan': 57753,
    'Huron': 59565,
    'Erie': 25655,
    'Ontario': 19009,
})

lakes['Surface Area (sq km)'] = surface_area
lakes

,Avg Depth (m),Max Depth (m),Surface Area (sq km)
Erie,19,64,25655
Huron,59,229,59565
Michigan,85,281,57753
Ontario,86,244,19009
Superior,149,406,82097


Notice how the index values allowed pandas to "align" the new data with the existing data!

It's also possible to create new columns from existing columns. Say for example we wanted a column to track the difference between the avg depth and max depth. We'll call this the "depth delta".

In [27]:
lakes['Depth Delta'] = lakes['Max Depth (m)'] - lakes['Avg Depth (m)']
lakes

,Avg Depth (m),Max Depth (m),Surface Area (sq km),Depth Delta
Erie,19,64,25655,45
Huron,59,229,59565,170
Michigan,85,281,57753,196
Ontario,86,244,19009,158
Superior,149,406,82097,257


## Data Indexing and Selection

Now that we can load data into pandas objects, we need to be able to access it. Pandas offers a variety of methods for accessing the data we need.

First, both `Series` and `DataFrame` objects support dictionary-style access with square brackets. Think of index label values as dictionary keys:

In [29]:
# We saw this above -- access a series like a dictionary to get a single value.
avg_depths['Michigan']

85

In [30]:
# DataFrame dictionary-style access returns the Series with that column index label:
lakes['Avg Depth (m)']

Erie         19
Huron        59
Michigan     85
Ontario      86
Superior    149
Name: Avg Depth (m), dtype: int64

Pandas also borrows array-style access from numpy. Namely, masking and "fancy indexing" work like in numpy.

In [32]:
# use a boolean mask to select just the items we want:
avg_depths[avg_depths > 60]

Michigan     85
Ontario      86
Superior    149
dtype: int64

In [33]:
# fancy indexing with an array of index labels:
avg_depths[['Erie', 'Ontario']]

Erie       19
Ontario    86
dtype: int64

In [38]:
# There is a potential problem with non-sequential integer indexes:
data_implicit = pd.Series([100, 200, 300, 400])
data_explicit = pd.Series([100, 200, 300, 400], index=[4, 9, 8, 1])
print("data_implicit[1] = {}\ndata_explicit[1] = {}".format(
    data_implicit[1],
    data_explicit[1]
))

data_implicit[1] = 200
data_explicit[1] = 400


To handle this potential confusion between label-based and position-based access and make data access easier in general, pandas provides three "indexers": `Series` and `DataFrame` attributes that expose differents ways to access the data.

- `iloc`: always integer position-based
- `loc`: always label-based
- `ix`: primarily label-based, falls back to position-based.

In [40]:
data_implicit.iloc[1]

200

In [49]:
data_explicit.iloc[1]

200

In [44]:
data_implicit.loc[4]

KeyError: 'the label [4] is not in the [index]'

In [45]:
data_explicit.loc[4]

100

In [52]:
# We can use slices to select more than one value as well. Here, get all values after the first one:
data_implicit.iloc[1:]

1    200
2    300
3    400
dtype: int64

In [48]:
# ix is useful with DataFrames and allows you to mix label and position-based
lakes.ix[0, ['Avg Depth (m)', 'Max Depth (m)']]

Avg Depth (m)    19
Max Depth (m)    64
Name: Erie, dtype: int64

In [73]:
# Pop quiz! Let's get all rows of the lakes dataframe except the last one:

In [74]:
# What about the first two rows and first two columns only?

In [75]:
lakes.loc['Erie']

Avg Depth (m)              19
Max Depth (m)              64
Surface Area (sq km)    25655
Depth Delta                45
Name: Erie, dtype: int64

`loc` accepts the following types of inputs:

- a single label (as above)
- a list or array of labels, e.g. ['a', 'b', 'c']
- a slice object with labels e.g. 'a':'c' (note that contrary to usual python slices, both the start and the stop are included!)
- A boolean array
- A callable function with one argument (the calling Series, DataFrame or Panel) and that returns valid output for indexing (one of the above)

`loc` and `iloc` also take an optional second parameter, the list of column names to return:

In [76]:
lakes.loc[['Michigan', 'Superior'], ['Max Depth (m)']]

,Max Depth (m)
Michigan,281
Superior,406


It is also possible to assign to the values at the locations you specify with the `iloc`, `loc`, or `ix` indexers! They aren't read-only.

In [77]:
df = pd.DataFrame(np.random.randint(0, 10, (3, 3)), columns=['A', 'B', 'C'])
df

,A,B,C
0,0,7,0
1,2,4,2
2,5,4,1


In [78]:
# Assign the value 100 to the 0,B and 1,B.
# Remember with label-based access, which `loc` uses, the high end of the slice is *included*.
df.loc[:1, 'B'] = 100
df

,A,B,C
0,0,100,0
1,2,100,2
2,5,4,1


### Examining Data

While you can manipulate and operate on your data in any way you can dream up, pandas does provide basic descriptive statistics and sorting functionality for you. I **highly** recommend reading the [Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/api.html#computations-descriptive-stats) to see what methods are available and save yourself some work!

The `describe` method is very useful with numeric data:

In [79]:
lakes.describe()

,Avg Depth (m),Max Depth (m),Surface Area (sq km),Depth Delta
count,5.000000,5.000000,5.000000,5.000000
mean,79.600000,244.800000,48815.800000,165.200000
std,47.389872,122.713895,26114.771207,77.296184
min,19.000000,64.000000,19009.000000,45.000000
25%,59.000000,229.000000,25655.000000,158.000000
50%,85.000000,244.000000,57753.000000,170.000000
75%,86.000000,281.000000,59565.000000,196.000000
max,149.000000,406.000000,82097.000000,257.000000


In [81]:
# We can get the highest value for a given Series with `max`:
max_depths.max()

406

In [85]:
# But what if we wanted the top 2? `sort_values` is the answer:
max_depths.sort_values(ascending=False).head(2)

Superior    406
Michigan    281
dtype: int64

In [87]:
# This is so common that there is actually a shortcut for it:
max_depths.nlargest(2)

Superior    406
Michigan    281
dtype: int64

In [92]:
# Which naturally works on DataFrames as well:
lakes.nlargest(2, 'Avg Depth (m)')

,Avg Depth (m),Max Depth (m),Surface Area (sq km),Depth Delta
Superior,149,406,82097,257
Ontario,86,244,19009,158


## Loading Data

Pandas provides a bunch of functions for reading data from a variety of sources, including CSV, Excel files, SQL databases, HDF5, even your computer's clipboard! The always-comprehensive pandas documentation has more info here: [https://pandas.pydata.org/pandas-docs/stable/io.html](https://pandas.pydata.org/pandas-docs/stable/io.html).

Let's read a local CSV dataset into a dataframe using the `read_csv` function.

In [94]:
df = pd.read_csv("data/Speed_Camera_Violations.csv")

This particular `DataFrame` contains speed camera violation data provided by the city of Chicago. This dataset is available at [https://catalog.data.gov/dataset/speed-camera-violations-997eb](https://catalog.data.gov/dataset/speed-camera-violations-997eb).

Let's start inspecting it by using the `head` method to look at the first five rows.

In [95]:
df.head()

,ADDRESS,CAMERA ID,VIOLATION DATE,VIOLATIONS,X COORDINATE,Y COORDINATE,LATITUDE,LONGITUDE,LOCATION
0,7738 S WESTERN,CHI065,07/08/2014,65,NaN,NaN,NaN,NaN,NaN
1,1111 N HUMBOLDT,CHI010,07/16/2014,56,NaN,NaN,NaN,NaN,NaN
2,5520 S WESTERN,CHI069,07/08/2014,10,NaN,NaN,NaN,NaN,NaN
3,1111 N HUMBOLDT,CHI010,07/26/2014,101,NaN,NaN,NaN,NaN,NaN
4,1111 N HUMBOLDT,CHI010,07/27/2014,92,NaN,NaN,NaN,NaN,NaN


When data is loaded from an external source, pandas will try to guess the datatype for each column. Let's see how it did:

In [102]:
pd.Series({col: df[col].dtype for col in df.columns})

ADDRESS            object
CAMERA ID          object
LATITUDE          float64
LOCATION           object
LONGITUDE         float64
VIOLATION DATE     object
VIOLATIONS          int64
X COORDINATE      float64
Y COORDINATE      float64
dtype: object

## Data types

Much of pandas functionality depends on the data types of the `Series` it's working with. For instance we can get summary measures and do numpy-like parallel operations on numeric types (`int64`, `float64`), or do date-based arithmetic with `date` series.

Notice above that the data type of the `VIOLATION DATE` column is "object", which, just like in numpy, means it is a generic type that isn't very useful. Let's turn those date strings into actual date objects, which are much better to work with.

In [103]:
# given a Series, pd.to_datetime returns a new Series with the string dates parsed as actual dates.
# We can then directly assign that Series back to the original column in our dataframe and pandas' magical Index
# functionality will make it all line up properly.
df["VIOLATION DATE"] = pd.to_datetime(df["VIOLATION DATE"], format="%m/%d/%Y")

df["VIOLATION DATE"].head()

0   2014-07-08
1   2014-07-16
2   2014-07-08
3   2014-07-26
4   2014-07-27
Name: VIOLATION DATE, dtype: datetime64[ns]

## Filtering

Now that we have a date column, we can do things like filter to only look at violations in 2015.

To do this, we'll create a "filter", essentially a boolean expression that works just like a mask or "fancy indexing" expression in numpy, and apply that filter to our dataframe to get just the rows we want.


In [105]:
import datetime

# note the extra parentheses below, these are necessary when creating a boolean filter expression with
# multiple comparisons like this
date_filter = ((df["VIOLATION DATE"] >= datetime.date(2015,1,1)) & (df["VIOLATION DATE"] < datetime.date(2016,1,1)))

# date_filter now contains a series of true/false values that we can use to extract just the values we are interested in
# by putting it in square brackets after the dataframe variable.
print(date_filter.head())
print()
print(date_filter.tail())

df_2015 = df[date_filter]

df_2015.head()

0    False
1    False
2    False
3    False
4    False
Name: VIOLATION DATE, dtype: bool

113991     True
113992     True
113993    False
113994     True
113995     True
Name: VIOLATION DATE, dtype: bool


,ADDRESS,CAMERA ID,VIOLATION DATE,VIOLATIONS,X COORDINATE,Y COORDINATE,LATITUDE,LONGITUDE,LOCATION
285,2912 W ROOSEVELT,CHI020,2015-01-07,8,1.157040e+06,1.894612e+06,41.866585,-87.698962,"(41.86658528313424, -87.69896237494204)"
286,5816 W JACKSON,CHI058,2015-01-07,22,1.137565e+06,1.898341e+06,41.877189,-87.770370,"(41.87718940463045, -87.7703695017756)"
287,57 E 95TH,CHI035,2015-01-07,4,1.178587e+06,1.841967e+06,41.721657,-87.621463,"(41.72165713317433, -87.62146289865515)"
288,324 S KEDZIE AVE,CHI123,2015-01-07,4,1.155058e+06,1.898247e+06,41.876599,-87.706143,"(41.87659894232968, -87.70614267856668)"
289,5471 W HIGGINS,CHI102,2015-01-07,14,1.139072e+06,1.931898e+06,41.969247,-87.764019,"(41.96924714863934, -87.76401945149458)"


This kind of filtering works for any kind of data type, provided you take care to make sure pandas is using the right data types for your data!

You may have noticed that many of the rows in this dataframe are missing lat/lon data. Pandas uses the "NaN" placeholder for missing data and offers some methods for dealing with it.

Both `Series` and `DataFrame` objects have `fillna` method that will replace missing data with a specified value.

In thise case however we may want to just drop those records that have missing data entirely:


In [106]:
df_no_nans = df.dropna(axis=0, how="any")
df_no_nans.head()

,ADDRESS,CAMERA ID,VIOLATION DATE,VIOLATIONS,X COORDINATE,Y COORDINATE,LATITUDE,LONGITUDE,LOCATION
148,2912 W ROOSEVELT,CHI020,2014-10-16,25,1.157040e+06,1.894612e+06,41.866585,-87.698962,"(41.86658528313424, -87.69896237494204)"
149,346 W 76th ST,CHI136,2014-10-16,4,1.175112e+06,1.854590e+06,41.756374,-87.633817,"(41.7563743554102, -87.6338165847716)"
150,11153 S VINCENNES,CHI022,2014-10-16,6,1.167029e+06,1.830594e+06,41.690702,-87.664122,"(41.690701951255015, -87.66412238501842)"
151,5454 W IRVING PARK,CHI050,2014-10-16,82,1.139043e+06,1.926097e+06,41.953330,-87.764267,"(41.95332954454448, -87.76426726425451)"
152,5446 W FULLERTON,CHI042,2014-10-16,38,1.139460e+06,1.915458e+06,41.924128,-87.762994,"(41.92412849125162, -87.76299399881007)"
